In [ ]:
!pip install datasets evaluate transformers
!pip install -U datasets huggingface_hub fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import evaluate

In [ ]:
# Read a FASTA file and return a list of sequences.

def read_fasta(file_path):
    sequences=[]
    current_sequence=[]
    with open(file_path,"r") as file:
        for line in file:
            if line.startswith(">"):
                if current_sequence:
                    sequences.append(''.join(current_sequence))
                    current_sequence=[]
            else:
                current_sequence.append(line.strip())
        if current_sequence:
            sequences.append(''.join(current_sequence))
    return sequences

In [ ]:
# Read the sequences from the FASTA files and create DataFrames for each dataset

sequences_do = read_fasta("clustered_fasta_files/do_transition_cdhit.fasta") # clustered MFIB+DIBS sequences by CD-HIT by 0.7
sequences_dd = read_fasta("clustered_fasta_files/dd_transition_cdhit.fasta") # clustered fuzdb sequences by CD-HIT by 0.7


df_do = pd.DataFrame(sequences_do, columns=["sequence"])
df_do["label"] = 0
df_dd = pd.DataFrame(sequences_dd, columns=["sequence"])
df_dd["label"] = 1


# Calculate the length of each sequence and add it as a new column

df_do["length"] = df_do["sequence"].apply(len)
df_dd["length"] = df_dd["sequence"].apply(len)

print("MFIB+DIBS:",df_do["length"].describe(), sep="\n")
print("fuzdb:",df_dd["length"].describe(), sep="\n")

# Filter out sequences longer than 1000 characters

print("Number of removed sequences:" ,df_dd[df_dd["length"] > 1000]["sequence"].count())
df_dd = df_dd[df_dd["length"] < 1001]

# Concatenate the DataFrames

df_all = pd.concat([df_do, df_dd], ignore_index=True)

# Remove all sequences that are repeated more than once

sequence_counts = df_all["sequence"].value_counts()
unique_sequences = sequence_counts[sequence_counts == 1].index
df_unique = df_all[df_all["sequence"].isin(unique_sequences)].reset_index(drop=True)
print("Unique sequences count:", df_unique.shape[0])

MFIB+DIBS:
count    614.000000
mean      49.609121
std       50.737725
min       11.000000
25%       16.000000
50%       32.000000
75%       70.000000
max      630.000000
Name: length, dtype: float64
fuzdb:
count     316.000000
mean      559.186709
std       594.045469
min        30.000000
25%       171.750000
50%       407.000000
75%       709.000000
max      4953.000000
Name: length, dtype: float64
Number of removed sequences: 42
Unique sequences count: 880


In [ ]:
# Split the dataset into training and testing sets, stratifying by label

train_df, test_df = train_test_split(df_unique, test_size=0.15, stratify=df_unique["label"], random_state=42, shuffle=True)

print("Train set size:", train_df.shape[0])
print("Test set size:", test_df.shape[0])
print("Train set label distribution:\n", train_df["label"].value_counts())
print("Test set label distribution:\n", test_df["label"].value_counts())
print("Train set length distribution:\n", train_df["length"].describe())
print("Test set length distribution:\n", test_df["length"].describe())
print("Columns in train set and test set:", train_df.columns.tolist(), test_df.columns.tolist())


Train set size: 748
Test set size: 132
Train set label distribution:
 label
0    518
1    230
Name: count, dtype: int64
Test set label distribution:
 label
0    92
1    40
Name: count, dtype: int64
Train set length distribution:
 count    748.000000
mean     155.312834
std      213.010061
min       11.000000
25%       22.000000
50%       64.000000
75%      153.250000
max      979.000000
Name: length, dtype: float64
Test set length distribution:
 count    132.000000
mean     133.265152
std      196.219415
min       11.000000
25%       18.000000
50%       54.500000
75%      136.750000
max      917.000000
Name: length, dtype: float64
Columns in train set and test set: ['sequence', 'label', 'length'] ['sequence', 'label', 'length']


In [ ]:
# load the tokenizer for the ESM-2 model

checkpoint = "facebook/esm2_t12_35M_UR50D"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, clean_up_tokenization_spaces=True)

# Function to tokenize the sequences in the dataset

def tokenize_function(example):
    return tokenizer(example["sequence"], truncation=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
# Convert the DataFrames to Hugging Face Datasets and tokenize them

train_dataset = Dataset.from_pandas(train_df, preserve_index=False)
train_dataset = train_dataset.map(tokenize_function, batched=True)

test_dataset = Dataset.from_pandas(test_df, preserve_index=False)
test_dataset = test_dataset.map(tokenize_function, batched=True)

print("Train datset:", train_dataset)
print("Test datset:", test_dataset)

Map:   0%|          | 0/748 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/132 [00:00<?, ? examples/s]

Train datset: Dataset({
    features: ['sequence', 'label', 'length', 'input_ids', 'attention_mask'],
    num_rows: 748
})
Test datset: Dataset({
    features: ['sequence', 'label', 'length', 'input_ids', 'attention_mask'],
    num_rows: 132
})


In [ ]:
# Remove unnecessary columns from the datasets

train_dataset = train_dataset.remove_columns(["sequence", "length"])
test_dataset = test_dataset.remove_columns(["sequence", "length"])

print("Train datset:", train_dataset)
print("Test datset:", test_dataset)

Train datset: Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 748
})
Test datset: Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 132
})


In [ ]:
##  Class Wighting to handle class imbalance

# Calculate Weights

label_counts = train_dataset.to_pandas()['label'].value_counts()
print(f"Label counts: {label_counts}")
num_samples_class_0 = label_counts[0]
num_samples_class_1 = label_counts[1]
total_samples = len(train_dataset)

weight_for_0 = total_samples / (2 * num_samples_class_0)
weight_for_1 = total_samples / (2 * num_samples_class_1)

class_weights = torch.tensor([weight_for_0, weight_for_1])

# Move to GPU if available and convert to float32

device = 'cuda' if torch.cuda.is_available() else 'cpu'
class_weights = class_weights.to(device).to(torch.float32)

print(f"Calculated class weights: {class_weights}")

batch_size = 4

# Step 2: Create a Custom Trainer

class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=batch_size, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Define the loss function with your calculated weights
        loss_fct = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

Label counts: label
0    518
1    230
Name: count, dtype: int64
Calculated class weights: tensor([0.7220, 1.6261], device='cuda:0')


In [ ]:
# Load the pre-trained model for sequence classification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# Set up the training arguments

model_name = checkpoint.split("/")[-1]
batch_size = 4

args = TrainingArguments(
    model_name,
    warmup_steps=0,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    report_to = "none",
)


# Load the metrics

f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

# Function to compute metrics during evaluation

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    results = {}
    # Use 'weighted' average to account for imbalance in the report
    results.update(f1_metric.compute(predictions=predictions, references=labels, average="weighted"))
    results.update(precision_metric.compute(predictions=predictions, references=labels, average="weighted"))
    results.update(recall_metric.compute(predictions=predictions, references=labels, average="weighted"))

    return results

# Create a Trainer instance with the model, training arguments, datasets, tokenizer, and metrics function

trainer = WeightedLossTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)


config.json:   0%|          | 0.00/778 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/136M [00:00<?, ?B/s]

Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t12_35M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Train the model using the Trainer instance

trainer.train()

Epoch,Training Loss,Validation Loss,F1,Precision,Recall
1,0.312700,0.183998,0.931571,0.931448,0.931818
2,0.145400,0.209407,0.938444,0.939604,0.939394
3,0.080700,0.196835,0.946362,0.946937,0.946970
4,0.057400,0.234371,0.938444,0.939604,0.939394
5,0.046800,0.283874,0.938444,0.939604,0.939394


TrainOutput(global_step=935, training_loss=0.12861075681798598, metrics={'train_runtime': 236.6519, 'train_samples_per_second': 15.804, 'train_steps_per_second': 3.951, 'total_flos': 292904834669784.0, 'train_loss': 0.12861075681798598, 'epoch': 5.0})

In [18]:
# Save model
model.save_pretrained("PLM_Transfer_Sequence_Outputs/my_model_dir")

In [16]:
# Save tokenizer
tokenizer.save_pretrained("PLM_Transfer_Sequence_Outputs/tokenizer_dir")

('PLM_Transfer_Sequence_Outputs/tokenizer_dir/tokenizer_config.json',
 'PLM_Transfer_Sequence_Outputs/tokenizer_dir/special_tokens_map.json',
 'PLM_Transfer_Sequence_Outputs/tokenizer_dir/vocab.txt',
 'PLM_Transfer_Sequence_Outputs/tokenizer_dir/added_tokens.json')

In [17]:
# Retrieve the log history from the trainer state

log_history = trainer.state.log_history
df_log = pd.DataFrame(log_history)

# Save the log history to a CSV file
df_log.to_csv("PLM_Transfer_Sequence_Outputs/training_logs.csv", index=False)